#### Wednesday, November 13, 2024

mamba activate rag_genai

This all runs in one pass. The code does not make any explicit reference to anything OpenAI yet a quick look at my usage for today reveals 17c of usage. I'm guessing LLamaIndex is using OpenAI under the hood, and if this is true, how can I control the model used?

# Constructing an Index-Based Deep Lake Vector Store for Semantic Search with LlamaIndex and OpenAI

copyright 2024, Denis Rothman

A Practical Guide to Building a Semantic Search Engine with Deep Lake, LlamaIndex, and OpenAI:

*   Installing the Environment
*   Creating and populating the Vector Store &   dataset
*   Getting started with  index-based semantic search




# Installing the environment

In [ ]:
#Google Drive option to store API Keys
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


*First run the following cells and restart Google Colab session if prompted. Then run the notebook again cell by cell to explore the code.*

In [ ]:
# !pip install llama-index-vector-stores-deeplake==0.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.9/608.9 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.3 MB/s eta 0:00:00

LlamaIndex supports Deep Lake vector stores through the DeepLakeVectorStore class.

In [ ]:
# !pip install deeplake==3.9.18

In [ ]:
# !pip install llama-index==0.10.64

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 8.0 MB/s eta 0:00:00


Next, let's import the required modules and set the needed environmental variables:

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

/home/rob/miniforge3/envs/rag_genai/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [ ]:
# !pip install sentence-transformers==3.0.1

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
#Retrieving and setting the OpenAI API key
# f = open("drive/MyDrive/files/api_key.txt", "r")
# API_KEY=f.readline().strip()
# f.close()

# #The OpenAI KeyActiveloop and OpenAI API keys
# import os
# import openai
# os.environ['OPENAI_API_KEY'] =API_KEY
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#Retrieving and setting the Activeloop API token
# f = open("drive/MyDrive/files/activeloop.txt", "r")
# API_token=f.readline().strip()
# f.close()
# ACTIVELOOP_TOKEN=API_token
# os.environ['ACTIVELOOP_TOKEN'] =ACTIVELOOP_TOKEN

In [ ]:
# For Google Colab and Activeloop while waiting for Activeloop (April 2024) pending new version
#This line writes the string "nameserver 8.8.8.8" to the file. This is specifying that the DNS server the system
#should use is at the IP address 8.8.8.8, which is one of Google's Public DNS servers.
# with open('/etc/resolv.conf', 'w') as file:
#    file.write("nameserver 8.8.8.8")

# Pipeline 1 : Collecting and preparing the documents

In [ ]:
# !mkdir data

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os

urls = [
    "https://github.com/VisDrone/VisDrone-Dataset",
    "https://paperswithcode.com/dataset/visdrone",
    "https://openaccess.thecvf.com/content_ECCVW_2018/papers/11133/Zhu_VisDrone-DET2018_The_Vision_Meets_Drone_Object_Detection_in_Image_Challenge_ECCVW_2018_paper.pdf",
    "https://github.com/VisDrone/VisDrone2018-MOT-toolkit",
    "https://en.wikipedia.org/wiki/Object_detection",
    "https://en.wikipedia.org/wiki/Computer_vision",
    "https://en.wikipedia.org/wiki/Convolutional_neural_network",
    "https://en.wikipedia.org/wiki/Unmanned_aerial_vehicle",
    "https://www.faa.gov/uas/",
    "https://www.tensorflow.org/",
    "https://pytorch.org/",
    "https://keras.io/",
    "https://arxiv.org/abs/1804.06985",
    "https://arxiv.org/abs/2202.11983",
    "https://motchallenge.net/",
    "http://www.cvlibs.net/datasets/kitti/",
    "https://www.dronedeploy.com/",
    "https://www.dji.com/",
    "https://arxiv.org/",
    "https://openaccess.thecvf.com/",
    "https://roboflow.com/",
    "https://www.kaggle.com/",
    "https://paperswithcode.com/",
    "https://github.com/"
]

In [3]:
import requests
import re
import os
from bs4 import BeautifulSoup

def clean_text(content):
    # Remove references and unwanted characters
    content = re.sub(r'\[\d+\]', '', content)   # Remove references
    content = re.sub(r'[^\w\s\.]', '', content)  # Remove punctuation (except periods)
    return content

def fetch_and_clean(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad responses (e.g., 404)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Prioritize "mw-parser-output" but fall back to "content" class if not found
        content = soup.find('div', {'class': 'mw-parser-output'}) or soup.find('div', {'id': 'content'})
        if content is None:
            return None

        # Remove specific sections, including nested ones
        for section_title in ['References', 'Bibliography', 'External links', 'See also', 'Notes']:
            section = content.find('span', id=section_title)
            while section:
                for sib in section.parent.find_next_siblings():
                    sib.decompose()
                section.parent.decompose()
                section = content.find('span', id=section_title)

        # Extract and clean text
        text = content.get_text(separator=' ', strip=True)
        text = clean_text(text)
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching content from {url}: {e}")
        return None  # Return None on error

In [4]:
# Directory to store the output files
output_dir = './data/'  # More descriptive name
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# Processing each URL (and skipping invalid ones)
for url in urls:
    article_name = url.split('/')[-1].replace('.html', '')  # Handle .html extension
    filename = os.path.join(output_dir, f"{article_name}.txt")

    clean_article_text = fetch_and_clean(url)
    if clean_article_text:  # Only write to file if content exists
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(clean_article_text)

# 14.3 s


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [6]:
print(f"Content(ones that were possible) written to files in the '{output_dir}' directory.")

Content(ones that were possible) written to files in the './data/' directory.


In [7]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()

In [8]:
documents[0]

Document(id_='795f7ec7-51ef-405d-8c05-f7a002b689ba', embedding=None, metadata={'file_path': '/home/rob/Data/Documents/Github/rkaunismaa/RAG-Driven-Generative-AI/Chapter03/data/1804.06985.txt', 'file_name': '1804.06985.txt', 'file_type': 'text/plain', 'file_size': 3798, 'creation_date': '2024-11-13', 'last_modified_date': '2024-11-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='High Energy Physics  Theory arXiv1804.06985 hepth Submitted on 19 Apr 2018 Title A Near Horizon Extreme Binary Black Hole Geometry Authors Jacob Ciafre  Maria J. Rodriguez View a PDF of the paper titled A Near Horizon Extreme Binary Black Hole Geometry by Jacob Ciafre and Maria J. Rodriguez View PDF Abstract A new solution of fourdimensional vacuum General Relativity is prese

# Pipeline 2 : Creating and populating a Deep Lake Vector Store

**Replace `hub://denis76/drone_v2` by your organization and dataset name**

In [9]:
from llama_index.core import StorageContext

In [10]:
# vector_store_path = "hub://denis76/drone_v2"
# dataset_path = "hub://denis76/drone_v2"

vector_store_path = "VectorStore"
dataset_path = vector_store_path

In [11]:
# overwrite=True will overwrite dataset, False will append it
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=True)

In [12]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [13]:
# Create an index over the documents
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

Uploading data to deeplake dataset.


100%|██████████| 87/87 [00:00<00:00, 708.23it/s]

Dataset(path='VectorStore', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (87, 1)      str     None   
 metadata     json      (87, 1)      str     None   
 embedding  embedding  (87, 1536)  float32   None   
    id        text      (87, 1)      str     None   


In [14]:
import deeplake
ds = deeplake.load(dataset_path)  # Load the dataset

VectorStore loaded successfully.



In [15]:
import json
import pandas as pd
import numpy as np

# Assuming 'ds' is your loaded Deep Lake dataset

# Create a dictionary to hold the data
data = {}

# Iterate through the tensors in the dataset
for tensor_name in ds.tensors:
    tensor_data = ds[tensor_name].numpy()

    # Check if the tensor is multi-dimensional
    if tensor_data.ndim > 1:
        # Flatten multi-dimensional tensors
        data[tensor_name] = [np.array(e).flatten().tolist() for e in tensor_data]
    else:
        # Convert 1D tensors directly to lists and decode text
        if tensor_name == "text":
            data[tensor_name] = [t.tobytes().decode('utf-8') if t else "" for t in tensor_data]
        else:
            data[tensor_name] = tensor_data.tolist()

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

In [18]:
# Function to display a selected record
def display_record(record_number):
    record = df.iloc[record_number]
    display_data = {
        "ID": record.get("id", "N/A"),
        "Metadata": record.get("metadata", "N/A"),
        "Text": record.get("text", "N/A"),
        "Embedding": record.get("embedding", "N/A")
    }

    # Print the ID
    print("ID:")
    print(display_data["ID"])
    print()

    # Print the metadata in a structured format
    print("Metadata:")
    metadata = display_data["Metadata"]
    if isinstance(metadata, list):
        for item in metadata:
            for key, value in item.items():
                print(f"{key}: {value}")
            print()
    else:
        print(metadata)
    print()

    # Print the text
    print("Text:")
    print(display_data["Text"])
    print()

    # Print the embedding
    print("Embedding:")
    print(display_data["Embedding"])
    print()



In [17]:
# Function call to display a record
rec = 0  # Replace with the desired record number
display_record(rec)

ID:
['d0de5799-f7a2-466b-a2a2-d8d08b9786d7']

Metadata:
file_path: /home/rob/Data/Documents/Github/rkaunismaa/RAG-Driven-Generative-AI/Chapter03/data/1804.06985.txt
file_name: 1804.06985.txt
file_type: text/plain
file_size: 3798
creation_date: 2024-11-13
last_modified_date: 2024-11-13
_node_content: {"id_": "d0de5799-f7a2-466b-a2a2-d8d08b9786d7", "embedding": null, "metadata": {"file_path": "/home/rob/Data/Documents/Github/rkaunismaa/RAG-Driven-Generative-AI/Chapter03/data/1804.06985.txt", "file_name": "1804.06985.txt", "file_type": "text/plain", "file_size": 3798, "creation_date": "2024-11-13", "last_modified_date": "2024-11-13"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "795f7ec7-51ef-405d-8c05-f7a002b689ba", "node_type": "4", "

# Original documents

In [19]:
# Ensure 'text' column is of type string
df['text'] = df['text'].astype(str)
# Create documents with IDs
documents = [Document(text=row['text'], doc_id=str(row['id'])) for _, row in df.iterrows()]

# Pipeline 3:Index-based RAG

## User input and RAG parameters

In [20]:
user_input="How do drones identify vehicles?"

#similarity_top_k
k=3
#temperature
temp=0.1
#num_output
mt=1024

## Cosine similarity metric

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def calculate_cosine_similarity_with_embeddings(text1, text2):
    embeddings1 = model.encode(text1)
    embeddings2 = model.encode(text2)
    similarity = cosine_similarity([embeddings1], [embeddings2])
    return similarity[0][0]

# Vector store index query engine

(The first type of llama_index we will be looking at is the VectorSoreStoreIndex. This type of index is useful when you have a collection of documents and you want to be able to query them. The VectorStoreIndex uses a vector store to store the embeddings of the documents and then uses these embeddings to perform similarity search.)

( VectorStoreIndex is a type of index within LlamaIndex that implements vector embeddings to represent and retrieve information from documents. These documents with similar meanings will have embeddings that are closer together in the vector space, as we explored in the previous chapter. However, this time, the VectorStoreIndex does not automatically use the existing Deep Lake vector store. It can create a new in-memory vector index, re-embed the documents, and create a new index structure.)

In [23]:
from llama_index.core import VectorStoreIndex
vector_store_index = VectorStoreIndex.from_documents(documents)

In [24]:
print(type(vector_store_index))

<class 'llama_index.core.indices.vector_store.base.VectorStoreIndex'>


(We now need a query engine to retrieve and synthesize the document(s) retrieved with an LLM—in our case, an OpenAI model)

In [25]:
vector_query_engine = vector_store_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [26]:
print(type(vector_query_engine))

<class 'llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine'>


## Query response and source

In [ ]:
import pandas as pd
import textwrap

def index_query(input_query):
    
    # We can now query the dataset and generate a response ...
    response = vector_query_engine.query(input_query)

    # Optional: Print a formatted view of the response (remove if you don't need it in the output)
    print(textwrap.fill(str(response), 100))

    node_data = []
    for node_with_score in response.source_nodes:
        node = node_with_score.node
        node_info = {
            'Node ID': node.id_,
            'Score': node_with_score.score,
            'Text': node.text
        }
        node_data.append(node_info)

    df = pd.DataFrame(node_data)

    # Instead of printing, return the DataFrame and the response object
    return df, response


We will evaluate the time it takes for the query to retrieve the relevant data and generate a response synthesis with the LLM (in this case, an OpenAI model). The output of the semantic search first returns a response synthesized by the LLM:

In [ ]:
import time

#start the timer
start_time = time.time()
df, response = index_query(user_input)
# Stop the timer
end_time = time.time()

# Calculate and print the execution time
elapsed_time = end_time - start_time

print(f"Query execution time: {elapsed_time:.4f} seconds")
print(df.to_markdown(index=False, numalign="left", stralign="left"))  # Display the DataFrame using markdown

Drones can identify vehicles using deep neural networks for visual detection and tracking. This
technology provides a performance benchmark for unmanned aerial vehicles in recognizing and
monitoring vehicles through video imagery.
Query execution time: 1.1018 seconds
| Node ID                              | Score    | Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Node information and relationships

The ID of the node guarantees full transparency and can be traced back to the original document, even when the index engines re-index the dataset. We can obtain the node source of the first node, for example, with the following code:

In [30]:
nodeid=response.source_nodes[0].node_id
nodeid

'7af5f202-f026-4241-bdd6-56952aa390ab'

We can drill down and retrieve the full text of the node containing the document that was synthesized by the LLM:

In [31]:
response.source_nodes[0].get_text()

"['ISSN 16747755 .  Peterman V. 26 August 2015. Landslide Activity Monitoring with the Help of Unmanned Aerial Vehicle . The International Archives of the Photogrammetry Remote Sensing and Spatial Information Sciences . XL1W4 215218. Bibcode  2015ISPAr.XL1..215P . doi  10.5194isprsarchivesXL1W42152015 . ISSN 16821750 .  Global Agriculture Drones and Robots Market Analysis  Forecast 20182028  ResearchAndMarkets.com . finance.yahoo.com . Archived from the original on 7 July 2019 . Retrieved 23 May 2019 .  Africa Farming Problems Aided With Drone Technology . Drone Addicts . 12 March 2018. Archived from the original on 29 June 2018 . Retrieved 23 May 2019 .  Drones That Launch Flaming Balls Are Being Tested To Help Fight Wildfires . NPR.org . Archived from the original on 25 April 2018 . Retrieved 5 April 2018 .  Old JM Lin S H Franklin MJM 2019. Mapping out barenosed wombat  Vombatus ursinus  burrows with the use of a drone. BMC Ecology. 1939. DOI 10.1186s1289801902575  Faust Daniel R. 2

## Optimized chunking

In [ ]:
# Assuming you have the 'response' object from query_engine.query()

for node_with_score in response.source_nodes:

    node = node_with_score.node  # Extract the Node object from NodeWithScore
    chunk_size = len(node.text)
    
    print(f"Node ID: {node.id_}, Chunk Size: {chunk_size} characters")

Node ID: 7af5f202-f026-4241-bdd6-56952aa390ab, Chunk Size: 3365 characters
Node ID: 23860a3a-f74e-41b2-9a7d-d4f2b686c104, Chunk Size: 4713 characters
Node ID: b5ddccde-2c64-4776-8098-72e351882656, Chunk Size: 3220 characters


## Performance metric

We will also implement a performance metric based on the accuracy of the queries and the time elapsed. This function calculates and prints a performance metric for a query, along with its execution time. The metric is based on the weighted average relevance scores of the retrieved information, divided by the time it took to get the results. Higher scores indicate better performance.

In [ ]:
import numpy as np

def info_metrics(response):
  
  # Calculate the performance (handling None scores)
  scores = [node.score for node in response.source_nodes if node.score is not None]
  
  if scores:  # Check if there are any valid scores
      weights = np.exp(scores) / np.sum(np.exp(scores))
      perf = np.average(scores, weights=weights) / elapsed_time
  else:
      perf = 0  # Or some other default value if all scores are None

  average_score=np.average(scores, weights=weights)
  print(f"Average score: {average_score:.4f}")
  print(f"Query execution time: {elapsed_time:.4f} seconds")
  print(f"Performance metric: {perf:.4f}")

In [34]:
info_metrics(response)

Average score: 0.8260
Query execution time: 1.1018 seconds
Performance metric: 0.7497


This performance metric is not an absolute value. It’s an indicator that we can use to compare this output with the other index engines. It may also vary from one run to another, due to the stochastic nature of machine learning algorithms. Additionally, the quality of the output depends on the user’s subjective perception. In any case, this metric will help compare the query engines’ performances in this chapter.

# Tree index query engine

(The second type of llama_index we will look as is the TreeIndex. This index engine is a hierarchical index that organizes data into a tree structure. It uses a tree-based data structure to store and retrieve data, which can improve query performance and efficiency.)

The tree index organizes documents in a tree structure, with broader summaries at higher levels and detailed information at lower levels. Each node in the tree summarizes the text it covers. The tree index is efficient for large datasets and queries large collections of documents rapidly by breaking them down into manageable optimized chunks. Thus, the optimization of the tree structure allows for rapid retrieval by traversing the relevant nodes without wasting time.

In [ ]:
from llama_index.core import TreeIndex
tree_index = TreeIndex.from_documents(documents)
# 33.2s

In [36]:
print(type(tree_index))

<class 'llama_index.core.indices.tree.base.TreeIndex'>


In [37]:
tree_query_engine = tree_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [38]:
import time
import textwrap
# Start the timer
start_time = time.time()
response = tree_query_engine.query(user_input)
# Stop the timer
end_time = time.time()
# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(response), 100))

Query execution time: 4.7330 seconds
Drones identify vehicles using computer vision techniques, which involve processing images or video
captured by the drone's camera. This process typically includes using convolutional neural networks
(CNNs) to analyze the visual data and recognize patterns that correspond to vehicles. By leveraging
CNNs, drones can detect and classify vehicles based on features learned during training, allowing
them to identify different types of vehicles with a high degree of accuracy.


## Performance metric

In [39]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.773
Query execution time: 4.7330 seconds
Performance metric: 0.1634


# List index query engine

In [40]:
from llama_index.core import ListIndex
list_index = ListIndex.from_documents(documents)

In [41]:
print(type(list_index))

<class 'llama_index.core.indices.list.base.SummaryIndex'>


In [42]:
list_query_engine = list_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [43]:
#start the timer
start_time = time.time()
response = list_query_engine.query(user_input)
# Stop the timer
end_time = time.time()
# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(response), 100))

Query execution time: 16.3979 seconds
Drones can identify vehicles through computer vision systems that process image data captured by
cameras mounted on the drones. These systems use algorithms to analyze the images, detect specific
objects like vehicles, and determine their locations within the scene. By recognizing patterns and
features in the images, drones can effectively identify vehicles as part of their visual
surveillance capabilities.


## Performance metric

In [44]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.800
Query execution time: 16.3979 seconds
Performance metric: 0.0488


# Keyword index query index

In [ ]:
from llama_index.core import KeywordTableIndex
keyword_index = KeywordTableIndex.from_documents(documents)

# 2m 20.1s

In [46]:
# Extract data for DataFrame
data = []
for keyword, doc_ids in keyword_index.index_struct.table.items():
    for doc_id in doc_ids:
        data.append({"Keyword": keyword, "Document ID": doc_id})

# Create the DataFrame
df = pd.DataFrame(data)
df

,Keyword,Document ID
0,kerr black holes,cc87361d-e2a2-4271-bbb7-166de3dcd632
1,black,cc87361d-e2a2-4271-bbb7-166de3dcd632
2,energy,cc87361d-e2a2-4271-bbb7-166de3dcd632
3,binary,cc87361d-e2a2-4271-bbb7-166de3dcd632
4,nhek black hole,cc87361d-e2a2-4271-bbb7-166de3dcd632
...,...,...
4993,intelligent drones,0c8e9428-232f-4454-9860-91e919b12c63
4994,range,0c8e9428-232f-4454-9860-91e919b12c63
4995,wikimedia,0c8e9428-232f-4454-9860-91e919b12c63
4996,future of warfare,0c8e9428-232f-4454-9860-91e919b12c63


In [47]:
keyword_query_engine = keyword_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [48]:
import time

# Start the timer
start_time = time.time()

# Execute the query (using .query() method)
response = keyword_query_engine.query(user_input)

# Stop the timer
end_time = time.time()

# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(response), 100))

Query execution time: 1.8592 seconds
Drones are used for various applications such as environmental monitoring, precision agriculture,
geological hazard studies, and more. They are equipped with sensors like radar, optical, and thermal
cameras to capture images and data. These sensors help drones in identifying vehicles by providing
visual information and data analysis capabilities.


## Performance metric

In [49]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.693
Query execution time: 1.8592 seconds
Performance metric: 0.3725
